<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Обозначения:

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
#импортируем нужные нам библиотеки и инструменты
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

In [2]:
#считываем датасеты
df_1 = pd.read_csv('/datasets/geo_data_0.csv')
df_2 = pd.read_csv('/datasets/geo_data_1.csv')
df_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
#создадим список с датасетами регионов
regions = [df_1, df_2, df_3]

#объявим константу
RND = 12345
state = RandomState(12345) 

#посмотрим на первые 5 строк датасетов
for region in regions:
    print(region.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [4]:
#посмотрим общую информацию 
for region in regions:
    print(region.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

Ни в одном из датафреймов трех регионов нет пропусков. Посмотрим есть ли в датафреймах явные дубликаты.

In [5]:
for region in regions:
    print(region.duplicated().sum())

0
0
0


Посмотрим на корреляцию параметров датафреймов

In [6]:
for region in regions:
    print(region.corr())

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


Во втором датафрейме замечена сильная связь между признаком f2 и целевым признаком. В остальных датайфреймах связь между этими признаками тоже выражена, но не так сильно, как во втором. Также в первом датафрейме есть слабая отрицательная корреляция между  признаками f1 и f0.

## Обучение и проверка модели

Напишем функцию по разделению на обучающую и валидационную выборки, обучению модели линейной регрессии и формированию предсказаний

In [7]:
def predict(region):
    features = region.drop(['id', 'product'], axis=1)
    target = region['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=RND) 
    model = LinearRegression()
    model.fit(features_train, target_train)
    return (model.predict(features_valid), target_valid)

Сохраним предсказания и правильные ответы на валидационной выборке

In [8]:
predictions_1, target_valid_1 = predict(df_1)
predictions_2, target_valid_2 = predict(df_2)
predictions_3, target_valid_3 = predict(df_3)

Выведем RMSE и средний запас предсказанного сырья по каждому региону

In [9]:
print('1ый регион: RMSE {:.2f}'.format(mean_squared_error(predictions_1, target_valid_1) ** 0.5), \
      ', средний запас предсказанного сырья {:.2f}'.format(predictions_1.mean()))
print('2ой регион: RMSE {:.2f}'.format(mean_squared_error(predictions_2, target_valid_2) ** 0.5), \
      ', средний запас предсказанного сырья {:.2f}'.format(predictions_2.mean()))
print('3ий регион: RMSE {:.2f}'.format(mean_squared_error(predictions_3, target_valid_3) ** 0.5), \
      ', средний запас предсказанного сырья {:.2f}'.format(predictions_3.mean()))

1ый регион: RMSE 37.58 , средний запас предсказанного сырья 92.59
2ой регион: RMSE 0.89 , средний запас предсказанного сырья 68.73
3ий регион: RMSE 40.03 , средний запас предсказанного сырья 94.97


У первого и третьего региона значения RMSE достаточно большие и почти одинаковые, а вот значение RMSE второго региона меньше 1. Исходя из этого делаем вывод, что модель линейной регресии лучше обучилась на предоставленных данных второго региона.

## Подготовка к расчёту прибыли

Создаем константы ключевых значений

In [10]:
BEST_WELLS = 200
BUDGET = 10 ** 10
PRICE_1K_BARREL = 450 * 10 ** 3

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [11]:
min_value = BUDGET / BEST_WELLS / PRICE_1K_BARREL
min_value

111.11111111111111

Сравним полученный объём сырья со средним запасом в каждом регионе

In [12]:
for region in regions:
    if min_value < region['product'].mean():
        print('Разработка в регионе будет прибыльной')
    else:
        print('Разработка в регионе будет убыточной')

Разработка в регионе будет убыточной
Разработка в регионе будет убыточной
Разработка в регионе будет убыточной


Исходя из этого получается, что в любом регионе разработка месторождений будет убыточной

Создадим функцию для подсчета прибыли

In [13]:
def revenue(predictions, target):
    predictions_sorted = pd.Series(predictions).reset_index(drop=True).sort_values(ascending=False)
    best_predictions_wells = predictions_sorted[:BEST_WELLS]
    sum_target = pd.Series(target).reset_index(drop=True)[best_predictions_wells.index].sum()
    revenue = (sum_target * PRICE_1K_BARREL - BUDGET) / 1000000000
    return revenue

Выведем прибыль для полученного объёма сырья для каждого региона

In [14]:
print('Прибыль для полученного объёма сырья 1 региона {:.2f} млрд'.format(revenue(predictions_1, target_valid_1)))
print('Прибыль для полученного объёма сырья 2 региона {:.2f} млрд'.format(revenue(predictions_2, target_valid_2)))
print('Прибыль для полученного объёма сырья 3 региона {:.2f} млрд'.format(revenue(predictions_3, target_valid_3)))

Прибыль для полученного объёма сырья 1 региона 3.32 млрд
Прибыль для полученного объёма сырья 2 региона 2.42 млрд
Прибыль для полученного объёма сырья 3 региона 2.71 млрд


По данным подсчетам самым прибыльным получился 1ый регион

In [15]:
target_valid_1

71751     10.038645
80493    114.551489
2655     132.603635
53233    169.072125
91141    122.325180
            ...    
12581    170.116726
18456     93.632175
73035    127.352259
63834     99.782700
43558    177.821022
Name: product, Length: 25000, dtype: float64

## Расчёт прибыли и рисков 

Напишем функцию с бутстрэпом в 1000 выборок, которая будет возвращать среднюю прибыль, 95%-й доверительный интервал и риск убытков по каждому региону

In [16]:
def bootstrap(target, predictions):
    values = []
    for i in range(1000):
        target_subsample = pd.Series(target).reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        pred_subsample = predictions[target_subsample.index] 
        values.append(revenue(pred_subsample, target_subsample))        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    risk = st.percentileofscore(values, 0)
    return mean, lower, upper, risk

Применим данную функцию

In [17]:
print('Информация по 1ому региону: Средняя прибыль: {:.2f}, \
Доверительный интервал: [{:.2f}, {:.2f}], Риск: {:.2f}%'.format(*bootstrap(target_valid_1, predictions_1)))
print('Информация по 2ому региону: Средняя прибыль: {:.2f}, \
Доверительный интервал: [{:.2f}, {:.2f}], Риск: {:.2f}%'.format(*bootstrap(target_valid_2, predictions_2)))
print('Информация по 3ему региону: Средняя прибыль: {:.2f}, \
Доверительный интервал: [{:.2f}, {:.2f}], Риск: {:.2f}%'.format(*bootstrap(target_valid_3, predictions_3)))

Информация по 1ому региону: Средняя прибыль: 0.40, Доверительный интервал: [-0.11, 0.91], Риск: 6.90%
Информация по 2ому региону: Средняя прибыль: 0.46, Доверительный интервал: [0.08, 0.86], Риск: 0.70%
Информация по 3ему региону: Средняя прибыль: 0.39, Доверительный интервал: [-0.11, 0.93], Риск: 6.50%


Вывод: многообещающим для разработки месторождений представляется 2ой регион поскольку риск убытка среди рассматриваемых регионов наименьший, а средняя прибыль - наибольшая